## `eigmax` of Custom Matrices

 

In [37]:
using LinearAlgebra
 
struct Custom
     D::Diagonal{Float64,Vector{Float64}}
     V::Vector{Float64}
end
Custom(v) = Custom(Diagonal(v), v)
matrix(c::Custom) = c.D + c.V*c.V'

# Function 
f(c::Custom, λ) = 1 + sum(c.V[i]^2 / (c.D.diag[i] - λ) for i=1:length(c.V))
 
# Derivative
f′(c::Custom, λ) = sum(c.V[i]^2 / (c.D.diag[i] - λ)^2 for i=1:length(c.V))
 
# Objective function to minimize
obj(f, c, x) = abs(0 - f(c, x))
 
# Newtons method
function newtons_method(c, f, f′, x0 = maximum(c.V)+0.1, tol = 1e-6; debug = false)
    δ = obj(f, c, x0)
    debug && println("δ = $δ")
    while δ > tol
        debug && println("δ = $δ")
        x0 = x0 - f(c, x0)/f′(c, x0)
        δ = obj(f, c, x0)
    end
    x0
end


newtons_method (generic function with 3 methods)

In [38]:
c = Custom([1.,2.,3.])

Custom([1.0 0.0 0.0; 0.0 2.0 0.0; 0.0 0.0 3.0], [1.0, 2.0, 3.0])

In [40]:
@time e1 = eigmax(matrix(c));

  0.000039 seconds (22 allocations: 2.625 KiB)


In [44]:
m = matrix(c)
@time maximum(eigvals(m));

  0.000021 seconds (16 allocations: 2.094 KiB)


In [21]:
@time e2 = newtons_method(c, f, f′)

  0.000008 seconds (75 allocations: 2.328 KiB)


16.597761829372512

In [48]:
c = Custom(rand(1000))

Custom([0.734554 0.0 … 0.0 0.0; 0.0 0.843931 … 0.0 0.0; … ; 0.0 0.0 … 0.466908 0.0; 0.0 0.0 … 0.0 0.610578], [0.734554, 0.843931, 0.397923, 0.490523, 0.669645, 0.881555, 0.413584, 0.503393, 0.349989, 0.682701  …  0.72782, 0.196977, 0.394431, 0.58802, 0.982493, 0.802634, 0.566481, 0.666212, 0.466908, 0.610578])

In [53]:
m = matrix(c)
@time e1 = eigmax(m)

  0.114202 seconds (17 allocations: 7.989 MiB)


332.697606834679

In [54]:
@time e2 = newtons_method(c, f, f′)

  0.098372 seconds (196.07 k allocations: 9.438 MiB, 5.07% gc time)


332.69760676084053